# Penn World Table in Python

**Author:** Luc Hens

**Version:** 27 January 2022

**Description:** Python code to read the Penn World Table and store the data in a dataframe called pwt. Examples of manipulations of series, subsetting of data, tables, graphs, and computations. Some of the examples show how to do the work for the research project for Introduction to Macroeconomics (*Inleiding tot de Macro-economie*).

Go to the Penn World Table home page (https://www.rug.nl/ggdc/productivity/pwt/) for the latest version of the Penn World Table. On 14 January 2022, the latest version was 10.0. The corresponding Stata data file was `pwt100.dta` (`pwt` stands for Penn World Table; `.dta` is the file extension for Stata data files). The path to the file was `https://www.rug.nl/ggdc/docs/pwt100.dta`.

When using the Penn World Table, make sure to document the source as explained in the "Attribution requirement" on https://www.rug.nl/ggdc/productivity/pwt/. 

The **documentation** (variable names,	variable definitions, units of measurement etc.) is stored in an Excel spreadsheet on https://www.rug.nl/ggdc/productivity/pwt/ (Info and Legend tabs). To extract this information from the spreadsheet (**warning**: this may take a while --be patient: 

In [ ]:
import pandas as pd
pwt_info   = pd.read_excel('https://www.rug.nl/ggdc/docs/pwt100.xlsx', sheet_name="Info")   # read information on pwt
pwt_legend = pd.read_excel('https://www.rug.nl/ggdc/docs/pwt100.xlsx', sheet_name="Legend") # read legend
# make sure that you use the most recent version (pwt100 refers to Penn World Table 10.0)
with pd.option_context("display.min_rows", 50, "display.max_rows", 100, \
                       "display.max_columns", 15, 'display.max_colwidth', 150):
    left_aligned_pwt_legend = pwt_legend.style.set_properties(**{'text-align': 'left'}) # .style.set_properties(**{'text-align': 'left'}) is to left-align columns when displaying
    display(left_aligned_pwt_legend) 

In [ ]:
# Display the information about the data set:
with pd.option_context("display.min_rows", 50, "display.max_rows", 100, \
                       "display.max_columns", 15, 'display.max_colwidth', 150):
    left_aligned_pwt_info = pwt_info.style.set_properties(**{'text-align': 'left'})  # to left-align columns when displaying
    display(left_aligned_pwt_info)

Read the Penn World Table using `pandas` and show the last lines (cases) of the data frame:

In [ ]:
import pandas as pd
pwt = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt100.dta').set_index('year')
# We use pwt as the name for the dataframe
# make sure that you use the most recent version (pwt100 refers to Penn World Table 10.0)
pwt.tail()   # show the last lines (cases) of the data frame

Let us verify what the **data types** of the variables (series) in the dataframe `pwt` are (`object` is a categorical variable; `int` is an integer; `float` is a number with decimals (real number)):

In [ ]:
pwt.dtypes  # show data types of the variables (series) in dataframe pwt

Another way to do get the data types (plus other information)  is:

In [ ]:
pwt.info()   # technical information about the dataframe

The `countrycode` and `country` variables are read as `object` type variables but are in fact categorical. Let is change the data type accordingly:

In [ ]:
pwt['countrycode'] = pwt['countrycode'].astype('category')
pwt['country']     = pwt['country'].astype('category')

(To check whether the data type has changed, you can run `pwt.info()` again.)

The Penn World Table is a panel data set: it combines time series data (yearly data starting in 1950) and cross-sectional data (countries). When the data file was read by pandas, we used `year` as the index for the dataframe. Let us check the data type of the index: 

In [ ]:
type(pwt.index)

As the index for the dataframe (`year`) refers to time, it is useful to change the format to the `datetime` data type:

In [ ]:
pwt.index = pd.to_datetime(pwt.index,format='%Y')
type(pwt.index)
pwt.tail()

## Change the variable name  of `pop`

The name of the population variable is `pop`. Because `pop` is also the name of the pandas function to drop an item from a data frame, we'll rename the variable (column) name `pop` to `population` using the `rename()` functiom from pandas (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pop.html;  https://www.geeksforgeeks.org/python-pandas-dataframe-rename/  )

In [ ]:
pwt.rename(columns = {"pop": "population"}, inplace = True)

In [ ]:
pwt.columns   # list(pwt) is another way of getting a list of the column names

In [ ]:
# To get a list of the country names without duplicates: (there must be a more elegant way to do this)
a = pwt.country.value_counts()
pd.set_option('display.max_rows', pwt.shape[0]+1)  # to display all rows of a
display(a)

In [ ]:
# Another way to get a list of the country names without duplicates: (there must be a more elegant way to do this)
# (see https://www.w3schools.com/python/python_howto_remove_duplicates.asp)
mylist = pwt["country"]
mylist = list(dict.fromkeys(mylist)) # Create a dictionary, using the List items as keys. This will automatically remove any duplicates because dictionaries cannot have duplicate keys.
display(mylist)

We are set now to start working with the `pwt` dataframe. If you plan to work with the Penn World Table regularly, it is useful to put all the `code` above (code is in the `grey` cells) in one code block.

## Add variables to the dataframe:

We create some additional variables: real GDP expressed in billions of US$ (rgdpe_in_billions), real GDP per person (`rgde_over_pop`), real GDP per worker (`rgde_over_emp`), the ratio of the labor force over the population (`emp_over_pop`): 

In [ ]:
import numpy as np
# First we create the series (columns) and assign the values with NaN ():
pwt = pwt.assign(rgdpe_in_billions = np.nan, rgdpe_over_pop = np.nan, 
                 rgdpe_over_emp = np.nan, cn_over_emp = np.nan, emp_over_pop = np.nan)
# Then we compute the values for the new series:
pwt.rgdpe_over_pop = pwt.rgdpe/pwt.population
pwt.rgdpe_over_emp = pwt.rgdpe/pwt.emp
pwt.emp_over_pop = pwt.emp/pwt.population   # employment as fraction of population
pwt.rgdpe_in_billions = pwt.rgdpe/10**3  
pwt.tail() # display the last lines of the new dataframe `pwt`

## Subsetting the data set: select one country
Subsetting in pandas: https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html 

Select only Belgium:

In [ ]:
BEL_data = pwt[(pwt['country'] == 'Belgium')]
BEL_data.head() # display the first lines of the new dataframe `BEL_data`

Select `rgdpe` for Belgium:

In [ ]:
y1 = pwt[(pwt['country'] == 'Belgium')].rgdpe
y1.head()

## Subsetting the data set: select variables 

Select the variables (columns, Series) we'll need: `country`, `rgdpe`, `population`, `emp`, `rgdpe_over_pop`, `rgdpe_over_emp`, `emp_over_pop`:

In [ ]:
BEL_data = BEL_data[["country", "rgdpe", "population", "emp", 
                    "rgdpe_over_pop", "rgdpe_over_emp", "emp_over_pop"]]
BEL_data.head() # display the first lines of the dataframe `BEL_data`

## Subsetting the data set:  select a single value

Display the row for Belgium in 2019:

In [ ]:
BEL_data.loc['2019-01-01']    

Here is another way of doing the same thing:

In [ ]:
pd.DataFrame(BEL_data, index=pd.date_range('2019-01-01', periods=1))    

By adding the argument `freq= 'YS'` (year start) and choosing a different value for `periods`, we can take a subset over several periods:

In [ ]:
pd.DataFrame(BEL_data, index=pd.date_range('2010-01-01', freq = 'YS', periods=5)) 

Display population (`rgdpe`, in millions) for Belgium in 2019:

In [ ]:
BEL_data.loc['2019-01-01'].rgdpe

Display labor force (`emp`, in millions) for Belgium in 2019:

In [ ]:
BEL_data.loc['2019-01-01'].emp

## Create time series (not needed because the index is of datetime type)

In [ ]:

pwt.tail()

## Time series diagram comparing income per person in two countries

In [ ]:
# We want to plot a time series diagram of rgdpe_over_pop for Belgium and India 
# First, we construct a dataframe that is a subsetting of pwt for Belgium:
BEL_data = pwt[(pwt['country'] == 'Belgium')] 
# Then limit the dataframe to just the variables `rgdpe_over_pop` and `rgdpe_over_emp`:
BEL_data = BEL_data[['rgdpe_over_pop','rgdpe_over_emp']]
# We do the same thing for India:
IND_data = pwt[(pwt['country'] == 'India')]
IND_data = IND_data[['rgdpe_over_pop','rgdpe_over_emp']]
IND_data.head()

Let us first illustrate how to make a **time series diagram** of income per person for one country (Belgium):

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(BEL_data['rgdpe_over_pop']) 
plt.box(False)   # get rid of the box
plt.ylabel('Income per person (PPP, 2017 US$)')    
plt.show()

Let us now make a time series diagram comparing income per person of Belgium and India:

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.box(False)   # get rid of the box
plt.ylabel('Income per person (PPP, 2017 US$)')    
# Plot a line chart:
plt.plot(BEL_data.rgdpe_over_pop, label='Belgium')
plt.plot(IND_data.rgdpe_over_pop, label='India')
plt.legend(edgecolor="None")    # adding legend to the curve (but is better to label the lines - see below)
plt.show() 

Labels next to the lines are better than a legend: 
( https://www.statology.org/matplotlib-add-text/: ` matplotlib.pyplot.text(x, y, s, fontdict=None)`)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.box(False)   # get rid of the box
plt.xlabel('')   # label on x-axis
plt.ylabel('Income per person (PPP, 2017 US$)')     
# Plot a line chart:
plt.plot(BEL_data.rgdpe_over_pop, label='Belgium')
plt.plot(IND_data.rgdpe_over_pop, label='India')
y_location_text_1 = BEL_data.loc['2019-01-01'].rgdpe_over_pop
y_location_text_2 = IND_data.loc['2019-01-01'].rgdpe_over_pop
x_location_text_1 = 18000  # I found this value by trial and error; still have to figure out why 18000 works  (12*2020 ?)
x_location_text_2 = 18000  # I found this value by trial and error; still have to figure out why 18000 works  (12*2020 ?)
plt.text(x_location_text_1, y_location_text_1, 'Belgium', horizontalalignment='left', verticalalignment='bottom')   # still to fix: replace 50000 by last value of variable
plt.text(x_location_text_1, y_location_text_2, 'India'  , horizontalalignment='left', verticalalignment='bottom')   # still to fix: replace 120000 by last value of variable
plt.show() 

In the research project for **Introduction to Macroeconomics**, this will be figure 1. If you can't get the diagram with the labels next to the lines, use a diagram with a legend.

To save the plot to a file in .pdf format:

In [ ]:
# paste the code from the previous code cell below:


fig.savefig('figure_1.pdf', dpi = 1000, transparent=True)  # dpi: dots per inch (the resolution of the image)
# the plot is saved to the current working directory (cwd)
# to find out what the current working directory (cwd) is:
import os
print('The file is saved to the current working directory: ',os.getcwd())

## Average annual growth rate of `rgdpe` between two years


To find the annual growth rate of `rgdpe` between two years economists use a formula using natural logarithms : $$   g \approx \frac{\ln x_{t+k}  - \ln x_t}{k}  $$
(for an explanation, see the assignment for the research project of Introduction to Macroeconomics)

In [ ]:
# BEL_data is the dataframe with the data for Belgium (see above). 

begin_year = 1950     # t   in the formula above
end_year   = 2019     # t+k in the formula above

begin_period = pd.to_datetime(begin_year, format='%Y') 
end_period   = pd.to_datetime(end_year  , format='%Y')

begin_value = BEL_data.loc[begin_period].rgdpe_over_pop
end_value   = BEL_data.loc[end_period].rgdpe_over_pop

# Here are the intermediate results
import numpy as np
print("value in " , begin_year, " : ", begin_value)
print("value in " , end_year,   " : ", end_value)
print("ln of value in " , begin_year, " : ", np.log(begin_value))
print("ln of value in " , end_year  , " : ", np.log(end_value))

# average annual growth rate:
g = (float(np.log(end_value)) - float(np.log(begin_value))) / (end_year - begin_year) 
# I had to add float() to make this work
print("growth rate (rounded): ", np.round(g,4))
# multiply by 100% to get the percentage growth rate:
print("The average annual growth rate of rgdpe between", 
      begin_year, "and" , end_year, "was" , np.round(g*100,2),"percent.")

## Scatter diagram of labor productivity and income per person

Make a **scatter plot** to investigate whether there is a (linear) relationship between labor productivity and real income per person:

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()   # to set the plot size:   plt.figure(figsize=(16, 8), dpi=150) 
plt.box(False)   # get rid of the box
# plt.title("text") # add title
plt.xlabel("Labor productivity (PPP, 2017 $)")   # add label to x-axis
plt.ylabel("Income per person (PPP, 2017 $)")  # add label to y-axis
plt.scatter(BEL_data.rgdpe_over_emp, BEL_data.rgdpe_over_pop)
plt.show()

# coefficient of correlation:
corr = BEL_data.rgdpe_over_emp.corr(BEL_data.rgdpe_over_pop)
import numpy as np
np.round(corr,4)  # round result to four decimals

It is straightforward to do the same for India. For the sake of comparison, combine the two scatter diagrams (figure 2 in the research project for Introduction to Macroeconomics):

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12, 6), dpi=1000)  
plt.subplot(1, 2, 1)   # 1 row, 2 columns, 1st plot
plt.box(False)
plt.title("(a) Belgium") # add title
plt.xlabel("Labor productivity (PPP, 2017 $)")   # add label to x-axis
plt.ylabel("Income per person (PPP, 2017 $)")  # add label to y-axis
plt.scatter(BEL_data.rgdpe_over_emp, BEL_data.rgdpe_over_pop)
plt.subplot(1, 2, 2)   # 1 row, 2 columns, 2nd plot
plt.box(False)
plt.title("(b) India") # add title
plt.xlabel("Labor productivity (PPP, 2017 $)")   # add label to x-axis
plt.ylabel("Income per person (PPP, 2017 $)")  # add label to y-axis
plt.scatter(IND_data.rgdpe_over_emp, IND_data.rgdpe_over_pop)
plt.show()

To save the plot to a file in .pdf format:

In [ ]:
# paste the code from the previous code cell below:

fig.savefig('figure_2.pdf', dpi = 1000, transparent=True)  # dpi: dots per inch (the resolution of the image)
# the plot is saved to the current working directory (cwd)
# to find out what the current working directory (cwd) is:
import os
print('The file is saved to the current working directory: ',os.getcwd())

**Coefficients of correlation** (research project for Introduction to Macroeconomics):

In [ ]:
corr_BEL = BEL_data.rgdpe_over_emp.corr(BEL_data.rgdpe_over_pop) # coefficient of correlation for Belgium
corr_IND = IND_data.rgdpe_over_emp.corr(IND_data.rgdpe_over_pop) # coefficient of correlation for India
import numpy as np
print("Belgium: ", np.round(corr_BEL,4),"     India: ", np.round(corr_IND,4))  # round result to four decimals